In [43]:
"""
Step 1

Getting the category and attributes from the excel file and matching it with the data from the json file.
"""
import json
import pandas as pd
from fuzzywuzzy import fuzz

def post_process_attribute_df(excel_file_path):
    print(excel_file_path)
    df = pd.read_csv(excel_file_path)
    df = df.transpose()
    df.head()
    df = df.drop("ID")
    df.columns = df.iloc[0]
    df = df.drop(["ATTRIBUTE"])
    df.reset_index(drop=True, inplace=True)
    df.fillna("empty", inplace = True)
    return df


with open('test.json', encoding='utf-8') as f:
    data = json.load(f)

def df_to_dict(df):
    category_dict = dict()
    df_dict = df.to_dict()
    for i in df_dict:
        category_dict[i] = list(df_dict[i].values())
    return category_dict




    
def post_process_category_dict(category_dict):
    for i in category_dict:
        category_dict[i] =  [x for x in category_dict[i] if x != 'empty']
    return category_dict


excel_file_path = 'Listed_Attribute/ATTRIBUTES.csv'
df=post_process_attribute_df(excel_file_path)

category_dict = df_to_dict(df)
category_dict = post_process_category_dict(category_dict)
print("done")

/home/sohaib/anaconda3/envs/NER/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Listed_Attribute/ATTRIBUTES.csv
done


In [56]:
data

Data


In [44]:
# import necessary libraries
'''

   Step 2
   
'''
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re
import copy


# download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english') + list(string.punctuation)

def preprocess(text):
    # tokenize text
    tokens = word_tokenize(text.lower())
    # remove stopwords and punctuation
    tokens = [token for token in tokens if token not in stop_words]
    # lemmatize words
    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # join tokens back into a string
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text



def extract_weight_and_length(text):
    """
    Extracts weight and length values along with their units from text using regular expressions.
    Returns a dictionary with keys 'weight' and 'length'.
    """
    # regular expressions to detect weight and length values
    weight_pattern = r"\b(\d+(?:\.\d+)?)\s*(g|kg|pound|lb)\b"
    length_pattern = r"\b(\d+(?:\.\d+)?)\s*(m|cm|mm|kilometer|km|mile|inch|in)\b"
    size_pattren =   r'\b(\d+\s*x\s*\d+)\s*(m|cm|mm|kilometer|km|mile|inch|in)\b'


    # extract weight and length values and their units from text
    weights = re.findall(weight_pattern, text, re.IGNORECASE)
    lengths = re.findall(length_pattern, text, re.IGNORECASE)
    sizes = re.findall(size_pattren, text, re.IGNORECASE)

    # convert weight values to grams
    size_dict = {}
    for size, unit in sizes:
        size_dict['size'] = str(size) + str(unit)

    # convert length values to meters
    length_dict = {}
    for length, unit in lengths:
        
        length_dict['length'] = str(length) + str(unit)

    # combine weight and length dictionaries and return
    result = {}
    result.update(length_dict)
    result.update(size_dict)
    return result

[nltk_data] Downloading package punkt to /home/sohaib/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/sohaib/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sohaib/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [45]:
'''

   Step 3
   
'''
import textwrap
from tqdm import tqdm
final_output = {}

def get_tags(data, category_dict):
    response = []
    final_json = dict()
    for index, chunk in tqdm(enumerate(data)):
        final_output = dict()
        description = chunk['SKU_SHORT_DESCRIPTION'] + chunk['SKU_LONG_DESCRIPTION'] + chunk['SKU_TITLE']
        description = preprocess(description)
        final_json["description"] = description
        final_json["SKU"] = chunk['SKU']
#         values  ['Jewelry']
#         category  CATEGORY_1
#         words  ['timelessness', 'symbol', '18kt', 'ø8', 'full', 'basic', 'production', 'cut', 'taste', 'castinggold', '0.03', 'sapphire', 'gold', '2', 'brilliant', 'earring', 'minimalist', 'tous', 'collection', 'technique', 'diamond', 'super-trendy', 'look', 'ct', 'h/si', 'yellow', 'le', 'hoop', 'create', 'blue', 'clasp', 'style', 'jewelry', 'thickness', 'good', 'combine', 'mm']
#         item  ('jewelry',)
#         word  timelessness
        for category in category_dict:
            values = category_dict[category]
            words = list(set(description.split(" ")))
            words = [word.lower() for word in words]
            for item in category_dict[category]:
                item = str(item)
                item = preprocess(item)
                
                for finder in words:
                    match_ratio = fuzz.ratio(item, finder)
                    if ( match_ratio > 90):
                        GEMSTONE_Token = True
                        if category in final_output:
                            per=fuzz.ratio('GEMSTONE_', category)
                            if per >= 50:
                                
                                
                                category_splited=category.split('_')
#                                 /* [0]GEMSTONE [1]4 [2]NAME */
#                                 /* GEMSTONE_1_NAME */
#                                 /*'GEMSTONE_1_NAME': ['diamond', 'sapphire'],*/
                                key=f"GEMSTONE_{category_splited[1]}_{category_splited[2]}"
#                                 /*GEMSTONE_4_Name*/
                                GEMSTONE_Token = True
                                if int(category_splited[1]) > 0:
                                    
                                    if final_output is not None:
                                        for gem_num in range(1,int(category_splited[1])+1):
                                            if final_output.get(f"GEMSTONE_{gem_num}_{category_splited[2]}") is not None:
                                                if item in final_output.get(f"GEMSTONE_{gem_num}_{category_splited[2]}"):
                                                    GEMSTONE_Token = False
                                                    continue
                                                else:
                                                    GEMSTONE_Token = False
                                                    continue

                                            else:
                                                GEMSTONE_Token = False
                                                final_output[category] = item
                                                continue
                            if GEMSTONE_Token:                                        
                                final_output[category].append(item)
                        else:
                            GEMSTONE_Token = True
                            per=fuzz.ratio('GEMSTONE_', category)
                            
                            if per >= 50:
#                                 /* [0]GEMSTONE [1]4 [2]NAME */
                                category_splited=category.split('_')
#                                 /*GEMSTONE_4_Name*/
                                key=f"GEMSTONE_{category_splited[1]}_{category_splited[2]}"
                                try:
                                    if int(category_splited[1]) > 0:
                                        if final_output is not None:
                                            for gem_num in range(1,int(category_splited[1])):
                                                if final_output.get(f"GEMSTONE_{gem_num}_{category_splited[2]}") is not None:
                                                    if item in final_output.get(f"GEMSTONE_{gem_num}_{category_splited[2]}"):
                                                        GEMSTONE_Token = False
                                                        continue
                                                else:
                                                    GEMSTONE_Token = False
                                                    continue
                                except:
                                    pass
                            if GEMSTONE_Token:
                                final_output[category] = [item]
        
                    if category in final_output and (finder == 'vermeil' or finder == 'Vermeil'):
                        final_output['Main_Material'] = "Silver"
                        final_output['Secondary_Material']= "Gold" 
                        final_output['Raw_Metal'] = "Silver"  
                        final_output['Plating_Metal'] = "Gold"
                    elif category in final_output and (finder == 'Silver Vermeil' or finder == 'silver vermeil'):
                        final_output['Main_Material'] = "Silver"
                        final_output['Secondary_Material']= "Gold" 
                        final_output['Raw_Metal'] = "Silver"  
                        final_output['Plating_Metal'] = "Gold"
                    elif finder == 'vermeil' or finder == 'Vermeil' or finder == 'Silver Vermeil' or finder == 'silver vermeil':
                        final_output['Main_Material'] = "Silver"
                        final_output['Secondary_Material']= "Gold" 
                        final_output['Raw_Metal'] = "Silver"  
                        final_output['Plating_Metal'] = "Gold"
                    if finder == 'diameter':
                        try:
                            diameter_li=words[words.index(finder):(words.index(finder)+3)]
                            if 'mm' in diameter_li:
                                final_output['PRODUCT_OUTTER_DIAMETER'] = diameter_li[1]+" "+diameter_li[2]
                        except:
                            pass
                    if finder == 'Gold Plated' or finder == 'Gold Filled':
                        if ('Silver' in description or 'silver' in description) and  category in final_output: 
                            final_output['Main_Material'] = "Silver"
                            final_output['Secondary_Material'] = "Gold" 
                            final_output['Raw_Metal'] = "Silver"
                            final_output['Plating_Metal'] = "Gold"
                        elif ('Silver' in description or 'silver' in description): 
                            final_output['Main_Material'] = "Silver"
                            final_output['Secondary_Material'] = "Gold" 
                            final_output['Raw_Metal'] = "Silver"
                            final_output['Plating_Metal'] = "Gold"
                        elif ('Brass' in description or 'brass' in description) and  category in final_output: 
                            final_output['Main_Material'] = "Brass"
                            final_output['Secondary_Material'] = "Gold"
                            final_output['Raw_Metal'] = "Brass"
                            final_output['Plating_Metal'] = "Gold"
                        elif ('Brass' in description or 'brass' in description): 
                            final_output['Main_Material'] = "Brass"
                            final_output['Secondary_Material'] = "Gold" 
                            final_output['Raw_Metal'] = "Brass"
                            final_output['Plating_Metal'] = "Gold"
                        elif ('Steel' in description or 'steel' in description) and  category in final_output: 
                            final_output['Main_Material'] = "Steel" 
                            final_output['Secondary_Material'] = "Gold" 
                            final_output['Raw_Metal'] = "Steel" 
                            final_output['Plating_Metal'] = "Gold"
                        elif ('Steel' in description or 'steel' in description): 
                            final_output['Main_Material'] = "Steel"
                            final_output['Secondary_Material'] = "Gold" 
                            final_output['Raw_Metal'] = "Steel"
                            final_output['Plating_Metal'] = "Gold"
                    if category in final_output and (finder == 'Sterling Silver' or finder == 'sterling silver'):
                        final_output['Main_Material'] = "Silver"
                        final_output['Raw_Material_Purity'] = "925"
                    elif finder == 'Sterling Silver' or finder == 'sterling silver':
                        final_output['Main_Material'] = "Silver"  
                        final_output['Raw_Material_Purity'] = "925"
                    if final_output.get('Secondary_Material') and  final_output['Secondary_Material'] == final_output['Main_Material']:
                        final_output['Raw_Metal'] = final_output['Main_Material']

        
        
        sizes = extract_weight_and_length(description)
        final_output.update(sizes)
        final_json["tags"] = final_output
        response.append(copy.deepcopy(final_json))
    return response
resp = get_tags(data, category_dict)
# 
# print("\n\nfinal output", final_output)




1600it [1:45:22,  3.95s/it]


In [17]:
resp

[{'description': 'le minimalist jewelry full style hoop earring gold\nblue sapphire diamond basic collection symbol\ntimelessness good taste combine earring create\nsuper-trendy look tous basic hoop earring 18kt\nyellow gold blue sapphire 0.03 ct h/si brilliant\ncut diamond thickness 2 mm ø8 mm hoop clasp\nproduction technique castinggold tous basic hoop\nearring blue sapphire diamond',
  'SKU': '212781640',
  'tags': {'CATEGORY_1': ['jewelry'],
   'CATEGORY_2': ['earring'],
   'CATEGORY_3': ['hoop'],
   'SKU_COLOR': ['blue'],
   'SKU_COLOR_SUBCATEGORY': ['blue', 'gold'],
   'RAW_METAL': ['gold'],
   'MAIN_MATERIAL': ['gold'],
   'SECONDARY_MATERIAL': ['gold'],
   'PLATING_METAL': ['gold'],
   'PRODUCT_SHAPE_MOTIFF_SUBCATEGORY': ['2'],
   'GEMSTONE_1_MAIN_CATEGORY': ['diamond'],
   'GEMSTONE_1_SUB_CATEGORY': ['diamond'],
   'GEMSTONE_1_CLASSIFICATION': ['diamond', 'sapphire'],
   'GEMSTONE_1_NAME': ['diamond', 'sapphire'],
   'GEMSTONE_1_COLOR': ['blue'],
   'length': '2mm'}}]

In [41]:
resp

[{'description': 'le minimalist jewelry full style hoop earring gold\nblue sapphire diamond basic collection symbol\ntimelessness good taste combine earring create\nsuper-trendy look tous basic hoop earring 18kt\nyellow gold blue sapphire 0.03 ct h/si brilliant\ncut diamond thickness 2 mm ø8 mm hoop clasp\nproduction technique castinggold tous basic hoop\nearring blue sapphire diamond',
  'SKU': '212781640',
  'tags': {'CATEGORY_1': ['jewelry'],
   'CATEGORY_2': ['earring'],
   'CATEGORY_3': ['hoop'],
   'SKU_COLOR': ['blue'],
   'SKU_COLOR_SUBCATEGORY': ['blue', 'gold'],
   'RAW_METAL': ['gold'],
   'MAIN_MATERIAL': ['gold'],
   'SECONDARY_MATERIAL': ['gold'],
   'PLATING_METAL': ['gold'],
   'PRODUCT_SHAPE_MOTIFF_SUBCATEGORY': ['2'],
   'GEMSTONE_1_MAIN_CATEGORY': ['diamond'],
   'GEMSTONE_1_SUB_CATEGORY': ['diamond'],
   'GEMSTONE_1_CLASSIFICATION': ['diamond'],
   'GEMSTONE_1_NAME': ['diamond'],
   'GEMSTONE_1_COLOR': ['blue'],
   'GEMSTONE_2_CLASSIFICATION': ['sapphire'],
   'GEMS

In [42]:
resp

[{'description': 'le minimalist jewelry full style hoop earring gold\nblue sapphire diamond basic collection symbol\ntimelessness good taste combine earring create\nsuper-trendy look tous basic hoop earring 18kt\nyellow gold blue sapphire 0.03 ct h/si brilliant\ncut diamond thickness 2 mm ø8 mm hoop clasp\nproduction technique castinggold tous basic hoop\nearring blue sapphire diamond',
  'SKU': '212781640',
  'tags': {'CATEGORY_1': ['jewelry'],
   'CATEGORY_2': ['earring'],
   'CATEGORY_3': ['hoop'],
   'SKU_COLOR': ['blue'],
   'SKU_COLOR_SUBCATEGORY': ['blue', 'gold'],
   'RAW_METAL': ['gold'],
   'MAIN_MATERIAL': ['gold'],
   'SECONDARY_MATERIAL': ['gold'],
   'PLATING_METAL': ['gold'],
   'PRODUCT_SHAPE_MOTIFF_SUBCATEGORY': ['2'],
   'GEMSTONE_1_MAIN_CATEGORY': ['diamond'],
   'GEMSTONE_1_SUB_CATEGORY': ['diamond'],
   'GEMSTONE_1_CLASSIFICATION': ['diamond'],
   'GEMSTONE_1_NAME': ['diamond'],
   'GEMSTONE_1_COLOR': ['blue'],
   'GEMSTONE_2_CLASSIFICATION': ['sapphire'],
   'GEMS

In [1]:
'''

   Step ...
   
   not use this under construction still
   
'''

import textwrap
from tqdm import tqdm
final_output = {}

def get_sku_tags(data, category_dict):
    response = []
    final_json = dict()
    for index, chunk in tqdm(enumerate(data)):
        final_output = dict()
        description = chunk['SKU_SHORT_DESCRIPTION'] + chunk['SKU_LONG_DESCRIPTION'] + chunk['SKU_TITLE']
        description = preprocess(description)
        wrapper = textwrap.TextWrapper(width=50, break_long_words=False, break_on_hyphens=False)
        description = wrapper.fill(description)
        final_json["description"] = description
        final_json["SKU"] = chunk['SKU']
        sizes = extract_weight_and_length(description)
        final_output.update(sizes)
        final_json["tags"] = final_output
        response.append(copy.deepcopy(final_json))
    return response
resp = get_tags(data, category_dict)
# 
# print("\n\nfinal output", final_output)

NameError: name 'data' is not defined

In [38]:
resp

[{'description': 'looking fun trendy look go iconic jewelry splash\ncolor lilac-colored elastic bracelet silver star\npart sweet doll collection versatile piece jewelry\ngo anywhere you.tous sweet doll lilac-colored\nelastic bracelet silver vermeil star charm charm\nsize 14 x 15 mm bracelet length 17\ncm.lilac-colored elastic sweet doll bracelet\nsilver vermeil star',
  'SKU': '212781640',
  'tags': {'length': '17cm', 'size': '14 x 15mm'}},
 {'description': 'looking fun trendy look go iconic jewelry splash\ncolor blue elastic bracelet silver star part sweet\ndoll collection versatile piece jewelry go\nanywhere you.tous sweet doll blue elastic bracelet\nsterling silver star charm charm size 14 x 15 mm\nbracelet length 17 cm.blue elastic sweet doll\nbracelet silver star',
  'SKU': '212781620',
  'tags': {'length': '17cm', 'size': '14 x 15mm'}},
 {'description': '-tous good vibe bracelet multicolored polyester\nviscose cord 23 mm bar 6 mm four-leaf clover\nsilver vermeil buckle sliding be

In [12]:
'''


   Step ...
   
   not use this under construction still
   

this cell doing some sort of preprocessing

variables:
         values  ['Jewelry']
         category  CATEGORY_1
         words  ['timelessness', 'symbol', '18kt', 'ø8', 'full', 'basic', 'production', 'cut', 'taste', 'castinggold', '0.03', 'sapphire', 'gold', '2', 'brilliant', 'earring', 'minimalist', 'tous', 'collection', 'technique', 'diamond', 'super-trendy', 'look', 'ct', 'h/si', 'yellow', 'le', 'hoop', 'create', 'blue', 'clasp', 'style', 'jewelry', 'thickness', 'good', 'combine', 'mm']
         item  ('jewelry',)
         word  timelessness

'''

index_count=0
for block in resp:
    
    for category in category_dict:
            values = category_dict[category]
            
            for item in category_dict[category]:
                item = str(item)
                item = preprocess(item)
                words = list(set(block['description'].split(" ")))
                words = [word.lower() for word in words]
                for words in words:
                    match_ratio = fuzz.ratio(item, word)
                    if ( match_ratio > 90):
                        GEMSTONE_Token = True
                        if category in block and (word == 'vermeil' or word == 'Vermeil'):
                            resp[index_count]['tags']['Main_Material'].append("Silver")
                            resp[index_count]['tags']['Secondary_Material'].append("Gold")
                            resp[index_count]['tags']['Raw_Metal'].append("Silver")
                            resp[index_count]['tags']['Plating_Metal'].append("Gold")
                        elif category in block and (word == 'Silver Vermeil' or word == 'silver vermeil'):
                            resp[index_count]['tags']['Main_Material'].append("Silver")
                            resp[index_count]['tags']['Secondary_Material'].append("Gold")
                            resp[index_count]['tags']['Raw_Metal'].append("Silver")
                            resp[index_count]['tags']['Plating_Metal'].append("Gold")
                        elif word == 'vermeil' or word == 'Vermeil' or word == 'Silver Vermeil' or word == 'silver vermeil':
                            resp[index_count]['tags']['Main_Material'] = "Silver"
                            resp[index_count]['tags']['Secondary_Material'] = "Gold"
                            resp[index_count]['tags']['Raw_Metal'] = "Silver"
                            resp[index_count]['tags']['Plating_Metal'] = "Gold"
                        elif category in block:
                            per=fuzz.ratio('GEMSTONE_', category)
                            if per>=67:
                                
                                category_splited=category.split('_')
                #                                 /* [0]GEMSTONE [1]4 [2]NAME */
                #                                 /* GEMSTONE_1_NAME */
                                key=f"GEMSTONE_{category_splited[1]}_{category_splited[2]}"
                #                                 /*GEMSTONE_4_Name*/
                                GEMSTONE_Token = True
                                if int(category_splited[1]) > 1:
                                    for gem_num in range(1,int(category_splited[1])):
                                        if item in resp[index_count]['tags'].get(f"GEMSTONE_{gem_num}_{category_splited[2]}"):
                                            GEMSTONE_Token = False
                                            continue
                                    if GEMSTONE_Token:
                                        resp[index_count]['tags'][category].append(item)
                                        continue
                            if GEMSTONE_Token:
                                resp[index_count]['tags'][category].append(item)
                        else:
                            per=fuzz.ratio('GEMSTONE_', category)
                            if per>=67:
                #                                 /* [0]GEMSTONE [1]4 [2]NAME */
                                category_splited=category.split('_')
                #                                 /*GEMSTONE_4_Name*/
                                key=f"GEMSTONE_{category_splited[1]}_{category_splited[2]}"
                                GEMSTONE_Token = True
                                if int(category_splited[1]) > 1:
                                    if resp[index_count] is not None:
                                        for gem_num in range(1,int(category_splited[1])):
                                            if resp[index_count]['tags'].get(f"GEMSTONE_{gem_num}_{category_splited[2]}") is not None:
                                                if item in resp[index_count]['tags'].get(f"GEMSTONE_{gem_num}_{category_splited[2]}"):
                                                    GEMSTONE_Token = False
                                                    continue    
                                        if GEMSTONE_Token:
                                            resp[index_count]['tags'][category] = [item]
                                            continue
                            if GEMSTONE_Token:
                                resp[index_count]['tags'][category] = [item]
                        if word == 'diameter':
                            diameter_li=words[words.index(word):(words.index(word)+3)]
                            if 'mm' in diameter_li:
                                print(diameter_li[1]+" "+diameter_li[2])
                                resp[index_count]['tags']['PRODUCT_OUTTER_DIAMETER'] = diameter_li[1]+" "+diameter_li[2]
                        if word == 'Gold Plated' or word == 'Gold Filled':
                            if ('Silver' in description or 'silver' in description) and  category in block: 
                                resp[index_count]['tags']['Main_Material'].append("Silver") 
                                resp[index_count]['tags']['Secondary_Material'].append("Gold") 
                                resp[index_count]['tags']['Raw_Metal'].append("Silver") 
                                resp[index_count]['tags']['Plating_Metal'].append("Gold")
                            elif ('Silver' in description or 'silver' in description): 
                                resp[index_count]['tags']['Main_Material'].append("Silver") 
                                resp[index_count]['tags']['Secondary_Material'].append("Gold") 
                                resp[index_count]['tags']['Raw_Metal'].append("Silver") 
                                resp[index_count]['tags']['Plating_Metal'].append("Gold")
                            elif ('Brass' in description or 'brass' in description) and  category in block: 
                                resp[index_count]['tags']['Main_Material'].append("Brass") 
                                resp[index_count]['tags']['Secondary_Material'].append("Gold") 
                                resp[index_count]['tags']['Raw_Metal'].append("Brass") 
                                resp[index_count]['tags']['Plating_Metal'].append("Gold")
                            elif ('Brass' in description or 'brass' in description): 
                                resp[index_count]['tags']['Main_Material'] = "Brass"
                                resp[index_count]['tags']['Secondary_Material'] = "Gold" 
                                resp[index_count]['tags']['Raw_Metal'] = "Brass"
                                resp[index_count]['tags']['Plating_Metal'] = "Gold"
                            elif ('Steel' in description or 'steel' in description) and  category in block: 
                                resp[index_count]['tags']['Main_Material'].append("Steel") 
                                resp[index_count]['tags']['Secondary_Material'].append("Gold") 
                                resp[index_count]['tags']['Raw_Metal'].append("Steel") 
                                resp[index_count]['tags']['Plating_Metal'].append("Gold")
                            elif ('Steel' in description or 'steel' in description): 
                                resp[index_count]['tags']['Main_Material'] = "Steel"
                                resp[index_count]['tags']['Secondary_Material'] = "Gold" 
                                resp[index_count]['tags']['Raw_Metal'] = "Steel"
                                resp[index_count]['tags']['Plating_Metal'] = "Gold"
                        if category in block and (word == 'Sterling Silver' or word == 'sterling silver'):
                            resp[index_count]['tags']['Main_Material'].append("Silver") 
                            resp[index_count]['tags']['Raw_Material_Purity'].append("925")
                        elif word == 'Sterling Silver' or word == 'sterling silver':
                            resp[index_count]['tags']['Main_Material'] = "Silver"  
                            resp[index_count]['tags']['Raw_Material_Purity'] = "925"
                        if resp[index_count]['tags'].get('Secondary_Material') and  resp[index_count]['tags']['Secondary_Material'] == resp[index_count]['tags']['Main_Material']:
                             resp[index_count]['tags']['Raw_Metal'] = resp[index_count]['tags']['Main_Material']
    print(index_count)
    index_count += 1
                                  
print('done')


NameError: name 'resp' is not defined

In [30]:
resp

[{'description': 'looking fun trendy look go iconic jewelry splash color lilac-colored elastic bracelet silver star part sweet doll collection versatile piece jewelry go anywhere you.tous sweet doll lilac-colored elastic bracelet silver vermeil star charm charm size 14 x 15 mm bracelet length 17 cm.lilac-colored elastic sweet doll bracelet silver vermeil star',
  'SKU': '212781640',
  'tags': {'CATEGORY_1': ['jewelry'],
   'CATEGORY_2': ['bracelet'],
   'CATEGORY_3': ['charm'],
   'RAW_METAL': ['silver'],
   'MAIN_MATERIAL': ['silver'],
   'SECONDARY_MATERIAL': ['silver'],
   'PRODUCT_SHAPE_MOTIFF_SUBCATEGORY': ['star'],
   'length': '17cm',
   'size': '14 x 15mm',
   'PRODUCT_SHAPE_MOTIFF_CATEGORY': 'celestial symbols'}},
 {'description': 'looking fun trendy look go iconic jewelry splash color blue elastic bracelet silver star part sweet doll collection versatile piece jewelry go anywhere you.tous sweet doll blue elastic bracelet sterling silver star charm charm size 14 x 15 mm bracel

In [46]:
''' 

   Step 4
   
'''
index_count=0
for block  in resp:
    
    if block:
        if block['tags'].get('RAW_METAL') is not None and'silver' in block['tags'].get('RAW_METAL'):
            if block['tags'].get('RAW_MATERIAL_PURITY') == 800:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = 'Second Law'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 925:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = 'Sterling'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 930:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = 'Argentium'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 958:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = 'Britannia'
        if block['tags'].get('RAW_METAL') is not None and 'platinum' in block['tags'].get('RAW_METAL'):
            if block['tags'].get('RAW_MATERIAL_PURITY') == 850:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = ''
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 900:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = ''
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 950:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = ''
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 999:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = ''
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 500:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = ''
        if block['tags'].get('RAW_METAL') is not None and 'palladium' in block['tags'].get('RAW_METAL'):
            if block['tags'].get('RAW_MATERIAL_PURITY') == 500:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = ''
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 950:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = ''
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 999:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = ''
        if block['tags'].get('RAW_METAL') is not None and 'gold' in block['tags'].get('RAW_METAL'):
            if block['tags'].get('RAW_MATERIAL_PURITY') == 333:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = '8k'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 375:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = '9k'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 416:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = '10k'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 416:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = '10k'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 417:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = '11k'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 458:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = '12k'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 500:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = '13k'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 541:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = '14k'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 583:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = '14k'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 585:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = '15k'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 625:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = '16k'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 666:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = '17k'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 708:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = '18k'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 750:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = '19k'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 791:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = '20k'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 833:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = '21k'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 875:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = '22k'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 916:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = '22k'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 917:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = '23k'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == 958:
                resp[index_count]['tags']['RAW_MATERIAL_STAMP'] = '24k' 
        
        if block['tags'].get('RAW_MATERIAL_STAMP') is not None: 
            if block['tags'].get('RAW_MATERIAL_STAMP') == 'Second Law':
                 resp[index_count]['tags']['RAW_METAL'] = 'Silver'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == 'Sterling':
                 resp[index_count]['tags']['RAW_METAL'] = 'Silver'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == 'Argentium':
                 resp[index_count]['tags']['RAW_METAL'] = 'Silver'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == 'Britannia':
                 resp[index_count]['tags']['RAW_METAL'] = 'Silver'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == '8k':
                 resp[index_count]['tags']['RAW_METAL'] = 'Gold'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == '9k':
                 resp[index_count]['tags']['RAW_METAL'] = 'Gold'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == '10k':
                 resp[index_count]['tags']['RAW_METAL'] = 'Gold'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == '10k':
                 resp[index_count]['tags']['RAW_METAL'] = 'Gold'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == '11k':
                 resp[index_count]['tags']['RAW_METAL'] = 'Gold'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == '12k':
                 resp[index_count]['tags']['RAW_METAL'] = 'Gold'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == '13k':
                 resp[index_count]['tags']['RAW_METAL'] = 'Gold'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == '14k':
                 resp[index_count]['tags']['RAW_METAL'] = 'Gold'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == '14k':
                 resp[index_count]['tags']['RAW_METAL'] = 'Gold'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == '15k':
                 resp[index_count]['tags']['RAW_METAL'] = 'Gold'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == '16k':
                 resp[index_count]['tags']['RAW_METAL'] = 'Gold'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == '17k':
                 resp[index_count]['tags']['RAW_METAL'] = 'Gold'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == '18k':
                 resp[index_count]['tags']['RAW_METAL'] = 'Gold'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == '19k':
                 resp[index_count]['tags']['RAW_METAL'] = 'Gold'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == '20k':
                 resp[index_count]['tags']['RAW_METAL'] = 'Gold'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == '21k':
                 resp[index_count]['tags']['RAW_METAL'] = 'Gold'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == '22k':
                 resp[index_count]['tags']['RAW_METAL'] = 'Gold'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == '22k':
                 resp[index_count]['tags']['RAW_METAL'] = 'Gold'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == '23k':
                 resp[index_count]['tags']['RAW_METAL'] = 'Gold'
            elif block['tags'].get('RAW_MATERIAL_STAMP') == '24k':
                 resp[index_count]['tags']['RAW_METAL'] = 'Gold'
        if block['tags'].get('RAW_MATERIAL_PURITY') is not None:
            if block['tags'].get('RAW_MATERIAL_PURITY') == '800':
                resp[index_count]['tags']['RAW_METAL'] = 'Silver'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == '925':
                resp[index_count]['tags']['RAW_METAL'] = 'Silver'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == '930':
                resp[index_count]['tags']['RAW_METAL'] = 'Silver'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == '958':
                resp[index_count]['tags']['RAW_METAL'] = 'Silver'
                
            elif block['tags'].get('RAW_MATERIAL_PURITY') == '850':
                resp[index_count]['tags']['RAW_METAL'] = 'Platinum'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == '900':
                resp[index_count]['tags']['RAW_METAL'] = 'Platinum'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == '950':
                resp[index_count]['tags']['RAW_METAL'] = 'Platinum'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == '999':
                resp[index_count]['tags']['RAW_METAL'] = 'Platinum'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == '500':
                resp[index_count]['tags']['RAW_METAL'] = 'Palladium'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == '950':
                resp[index_count]['tags']['RAW_METAL'] = 'Palladium'
            elif block['tags'].get('RAW_MATERIAL_PURITY') == '999':
                resp[index_count]['tags']['RAW_METAL'] = 'Palladium'
 

    
    index_count+=1

In [122]:
resp

[{'description': 'looking fun trendy look go iconic jewelry splash color lilac-colored elastic bracelet silver star part sweet doll collection versatile piece jewelry go anywhere you.tous sweet doll lilac-colored elastic bracelet silver vermeil star charm charm size 14 x 15 mm bracelet length 17 cm.lilac-colored elastic sweet doll bracelet silver vermeil star',
  'SKU': '212781640',
  'tags': {'CATEGORY_1': ['jewelry'],
   'CATEGORY_2': ['bracelet'],
   'CATEGORY_3': ['charm'],
   'RAW_METAL': ['silver'],
   'MAIN_MATERIAL': ['silver'],
   'SECONDARY_MATERIAL': ['silver'],
   'PRODUCT_SHAPE_MOTIFF_SUBCATEGORY': ['star'],
   'length': '17cm',
   'size': '14 x 15mm',
   'PRODUCT_SHAPE_MOTIFF_CATEGORY': 'celestial symbols'}},
 {'description': 'looking fun trendy look go iconic jewelry splash color blue elastic bracelet silver star part sweet doll collection versatile piece jewelry go anywhere you.tous sweet doll blue elastic bracelet sterling silver star charm charm size 14 x 15 mm bracel

In [47]:
'''
   Step 5
'''
CATEGORY_1 = ['jewelry']

CATEGORY_2= ['accesories', 'accesories', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'bracelet', 'brooches', 'brooches', 'brooches', 'brooches', 'earring', 'earring', 'earring', 'earring', 'earring', 'earring', 'earring', 'earring', 'earring', 'earring', 'earring', 'earring', 'earring', 'earring', 'earring', 'earring', 'earring', 'earring', 'earring', 'earring', 'earring', 'earring', 'earring', 'earring', 'necklace', 'necklace', 'necklace', 'necklace', 'necklace', 'necklace', 'necklace', 'necklace', 'necklace', 'necklace', 'necklace', 'necklace', 'necklace', 'necklace', 'necklace', 'necklace', 'necklace', 'necklace', 'necklace', 'pack & sets', 'pack & sets', 'pack & sets', 'pack & sets', 'pendants', 'pendants', 'rings', 'rings', 'rings', 'rings', 'rings', 'rings', 'rings', 'rings', 'rings', 'rings', 'rings', 'rings', 'rings', 'rings', 'rings', 'rings', 'rings', 'rings', 'rings', 'rings', 'rings', 'rings', 'rings', 'rings', 'rings','bracelet']

CATEGORY_3 = ['jewel extension', 'earring pressure', 'ankle', 'bangle', 'cuff', 'cable bracelet', 'elastic bracelet', 'mesh bracelet', 'cord bracelet', 'hinged', 'crossover', 'stretchy', 'open bracelet', 'statement', 'chain bracelet', 'friendship', 'slider', 'tipped', 'link', 'identification', 'charm', 'wrap', 'affirmation', 'beaded bracelet', 'strands', 'tennis / riviere bracelet', 'zº', 'bolo', 'brooches', 'dress-clips', 'fur-clips', 'jewelry-clips', 'ball', 'button', 'stud', 'hoop', 'drop', 'dangle', 'cluster', 'jacket', 'huggies', 'chandeliers', 'tear drop', 'barbell', 'trheader', 'cuff', 'chain earring', 'bajoran', 'c hoop', 'tassel', 'crawler', 'cartilage', 'two-sided', 'lobe', 'beaded earring', 'charm earring', 'chain necklace', 'beaded necklace', 'riviere necklace', 'short pendant', 'solitaire pendant', 'lariat', 'strand', 'thread necklace', 'bib necklace', 'rosary', 'twisted necklace', 'charm necklace', 'long pendant', 'locket', 'lavalier', 'torque', 'sautoir', 'pendant', 'choker', 'earring-sets', 'bracelet-sets', 'necklace-sets', 'ring-set', 'pendant-enhancers', 'pendant_slides', 'band', 'wedding band', 'two-finger', 'cocktail', 'engagement', 'triple', 'open-ring', 'nail', 'claddagh', 'antique', 'cluster', 'beaded ring', 'nugget', 'spinner', 'stack', 'graphic', 'signet', 'thumb', 'crossover', 'wrap', 'bypass', 'chain ring', 'charm ring', 'solitaire', 'college','cord']


index_count=0
for block  in resp:
    try:
        if block:
            if block['tags'].get('CATEGORY_3') is not None:
                if block['tags'].get('CATEGORY_2') is not None:
                    cat_3_val = block['tags'].get('CATEGORY_3')
                    for val in cat_3_val:
                        index = CATEGORY_3.index(val)
                        cat_2_val = CATEGORY_2[index]
                        if cat_2_val in block['tags'].get('CATEGORY_2'):
                            if block['tags'].get('CATEGORY_1') is None:
                                resp[index_count]['tags']['CATEGORY_1'] = 'jewelry'
                        else:
                            resp[index_count]['tags']['CATEGORY_2'].append(cat_2_val)
                            resp[index_count]['tags']['CATEGORY_2'] = list( dict.fromkeys(resp[index_count]['tags']['CATEGORY_2']) )
                            print(cat_2_val)
                            if block['tags'].get('CATEGORY_1') is None:
                                resp[index_count]['tags']['CATEGORY_1'] = 'jewelry'
                else:
                    cat_3_val = block['tags'].get('CATEGORY_3')
                    for val in cat_3_val:
                        index = CATEGORY_3.index(val)
                        print(val)
                        cat_2_val = CATEGORY_2[index]
                        print(cat_2_val)
                        resp[index_count]['tags']['CATEGORY_2'] = cat_2_val
                        if block['tags'].get('CATEGORY_1') is None:
                            resp[index_count]['tags']['CATEGORY_1'] = 'jewelry'

            elif block['tags'].get('CATEGORY_2') is not None:
                if block['tags'].get('CATEGORY_1') is None:
                    resp[index_count]['tags']['CATEGORY_1'] = 'jewelry'
    except:
        pass

    index_count+=1

print("Done")
    
    

necklace
necklace
necklace
necklace
choker
necklace
necklace
necklace
necklace
bracelet
necklace
choker
necklace
necklace
necklace
necklace
necklace
necklace
necklace
necklace
necklace
necklace
necklace
bracelet
bracelet
necklace
necklace
necklace
necklace
necklace
necklace
bracelet
necklace
necklace
necklace
necklace
bracelet
necklace
necklace
ball
earring
choker
necklace
necklace
necklace
bracelet
necklace
necklace
necklace
necklace
necklace
necklace
necklace
necklace
necklace
necklace
necklace
necklace
necklace
necklace
necklace
necklace
necklace
necklace
necklace
necklace
necklace
bracelet
necklace
necklace
necklace
necklace
necklace
earring
necklace
bracelet
necklace
necklace
necklace
necklace
necklace
necklace
necklace
bracelet
necklace
necklace
bracelet
necklace
necklace
bracelet
necklace
bracelet
necklace
necklace
necklace
necklace
bracelet
necklace
necklace
necklace
bracelet
necklace
bracelet
necklace
necklace
necklace
necklace
necklace
necklace
necklace
necklace
necklace
neck

In [48]:
'''Step 6 '''

product_shape_motiff_category = ['sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'celestial symbols', 'celestial symbols', 'celestial symbols', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'animals', 'love symbols', 'love symbols', 'love symbols', 'love symbols', 'birthstones', 'birthstones', 'birthstones', 'birthstones', 'birthstones', 'birthstones', 'birthstones', 'birthstones', 'birthstones', 'birthstones', 'birthstones', 'birthstones', 'nautical', 'nautical', 'nautical', 'nautical', 'nautical', 'nautical', 'popular characters', 'popular characters', 'popular characters', 'good luck symbols', 'good luck symbols', 'popular symbols', 'popular symbols', 'popular symbols', 'popular symbols', 'popular symbols', 'popular symbols', 'musical symbols', 'religious symbols', 'religious symbols', 'religious symbols', 'religious symbols', 'religious symbols', 'religious symbols', 'religious symbols', 'religious symbols', 'religious symbols', 'religious symbols', 'religious symbols', 'religious symbols', 'religious symbols', 'religious symbols', 'religious symbols', 'religious symbols', 'religious symbols', 'religious symbols', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'alphabet letters', 'numbers', 'numbers', 'numbers', 'numbers', 'numbers', 'numbers', 'numbers', 'numbers', 'numbers', 'numbers', 'zodiac-signs', 'zodiac-signs', 'zodiac-signs', 'zodiac-signs', 'zodiac-signs', 'zodiac-signs', 'zodiac-signs', 'zodiac-signs', 'zodiac-signs', 'zodiac-signs', 'zodiac-signs', 'zodiac-signs', 'supernatural-beings', 'supernatural-beings', 'supernatural-beings', 'supernatural-beings', 'supernatural-beings', 'supernatural-beings', 'supernatural-beings', 'supernatural-beings', 'flowers', 'flowers', 'flowers', 'flowers', 'flowers', 'flowers', 'flowers', 'flowers', 'flowers', 'flowers', 'flowers', 'flowers', 'flowers', 'musical instruments', 'musical instruments', 'musical instruments', 'musical instruments', 'musical instruments', 'musical instruments', 'musical instruments', 'musical instruments', 'musical instruments', 'musical instruments', 'musical instruments', 'musical instruments', 'musical instruments', 'musical instruments', 'occasions', 'occasions', 'occasions', 'occasions', 'occasions', 'occasions', 'occasions', 'occasions', 'occasions', 'occasions', 'occasions', 'occasions', 'occasions', 'occasions', 'occasions', 'occasions']

product_shape_motiff_subcategory = ['aerobics', 'airsoft', 'archery', 'baseball', 'basketball', 'billiards', 'boating', 'canoeing', 'kayaking', 'sailing', 'bowling', 'boxing', 'camping', 'cheerleading', 'climbing', 'ice-climbing', 'mountain-climbing', 'rock-climbing', 'cricket', 'curling', 'cycling', 'dart-games', 'disc-sports', 'equestrian', 'exercise-and-fitness', 'running', 'strength-training', 'walking', 'yoga', 'field-hockey', 'fishing', 'fly-fishing', 'ice-fishing', 'football', 'golf', 'gymnastics', 'hiking', 'hockey', 'hunting-and-shooting', 'ice-skating', 'inline-skating', 'lacrosse', 'lawn-games', 'bocce', 'croquet', 'horseshoes', 'pickle-ball', 'tetherball', 'martial-arts', 'hapkido', 'judo', 'karate', 'kung-fu', 'moo-duk-kwan', 'ninja', 'tae-kwon-do', 'tang-soo-do', 'paintball', 'racquetball', 'roller-skating', 'rugby', 'skateboarding', 'soccer', 'softball', 'squash', 'table-games', 'air-hockey', 'dome-hockey', 'foosball', 'shuffleboard', 'skee-ball', 'table-tennis', 'tennis', 'track-and-field', 'volleyball', 'water-sports', 'diving', 'kitesurfing', 'surfing', 'swimming', 'wakeboarding', 'waterskiing', 'windsurfing', 'winter-sports', 'sledding', 'snow-skiing', 'snowmobiling', 'snowshoeing', 'wrestling', 'moon', 'star', 'sun', 'alligators', 'bear', 'birds', 'bluebirds', 'canaries', 'doves', 'ducks', 'eagles', 'finches', 'flamingos', 'hummingbirds', 'owls', 'parrots', 'penguins', 'purple-martins', 'storks', 'swans', 'wrens', 'cattle', 'dinosaurs', 'dogs', 'dolphins', 'elephants', 'fish', 'foxes', 'frogs', 'giraffes', 'horses', 'house-cats', 'insects', 'bees', 'beetles', 'butterflies', 'dragonflies', 'ladybugs', 'kangaroos', 'leopards', 'lions', 'lizards', 'mice', 'monkeys', 'panthers', 'pigs', 'rabbits', 'sheep', 'snails', 'snakes', 'spiders', 'squirrels', 'tigers', 'turtles', 'whales', 'zebras', 'animal', 'cupid', 'heart', 'lip', 'locket', 'april-birthstones', 'august-birthstones', 'december-birthstones', 'february-birthstones', 'january-birthstones', 'july-birthstones', 'june-birthstones', 'march-birthstones', 'may-birthstones', 'november-birthstones', 'october-birthstones', 'september-birthstones', 'anchors', 'boat', 'canoes', 'kayaks', 'rowboats', 'sailboats', 'cartoon', 'fictional', 'superheroes', 'four leaf clovers', 'horshoe ', 'crosses', 'crowns', 'dollar', 'eight-balls', 'skeletons', 'skulls', 'musical note', 'buddhist-symbols', 'christian-symbols', 'christian-crosses', 'christian-fish', 'crucifixes', 'jesus-depictions', 'nativity-theme', 'saint-depictions', 'earth-based-religious-symbols', 'hindu-symbols', 'islamic-symbols', 'jewish-symbols', 'chai-hebrew-symbol', 'hamsa', 'menorahs', 'mezuzahs', 'star-of-david', 'torah', 'letter a', 'letter b', 'letter c', 'letter d', 'letter e', 'letter f', 'letter g', 'letter h', 'letter i', 'letter j', 'letter k', 'letter l', 'letter m', 'letter n', 'letter ñ', 'letter o', 'letter p', 'letter q', 'letter r', 'letter s', 'letter t', 'letter u', 'letter v', 'letter w', 'letter x', 'letter y', 'letter z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'aquarius', 'aries', 'cancer', 'capricorn', 'gemini', 'leo', 'libra', 'pisces', 'sagittarus', 'scorpio', 'taurus', 'virgo', 'aliens', 'angels', 'dragons', 'ghosts', 'mermaids', 'monsters', 'unicorns', 'wizards', 'carnation-flowers', 'chrysanthemum-flowers', 'daffodil-flowers', 'daisy-flowers', 'iris-flowers', 'lily-flowers', 'orchid-flowers', 'rose-flowers', 'sunflower-flowers', 'tulip-flowers', 'keyboard instrument', 'plant', 'flower', 'banjos', 'cellos', 'guitars', 'mandolins', 'violins', 'bassoons', 'clarinets', 'french-horns', 'harmonicas', 'saxophones', 'trombones', 'trumpets', 'tubas', 'piano', 'anniversary', 'birthday', 'graduation', 'holiday', 'christmas', 'easter', 'fathers-day', 'fourth-of-july', 'halloween', 'hanukkah', 'kwanzaa', 'mothers-day', 'st-patricks-day', 'thanksgiving', 'valentines-day', 'wedding']

index_count=0
for block  in resp:
    try:
        if block:

            if block['tags'].get('PRODUCT_SHAPE_MOTIFF_SUBCATEGORY') is not None:
                product_shape_motiff_subcategory_val = block['tags'].get('PRODUCT_SHAPE_MOTIFF_SUBCATEGORY')
                for element in block['tags'].get('PRODUCT_SHAPE_MOTIFF_SUBCATEGORY').copy():
                    print(element)
                    if element.isdigit():
                        block['tags'].get('PRODUCT_SHAPE_MOTIFF_SUBCATEGORY').remove(element)
                for val in product_shape_motiff_subcategory_val:
                    index = product_shape_motiff_subcategory.index(val)
                    product_shape_motiff_category_val = product_shape_motiff_category[index]
                    resp[index_count]['tags']['PRODUCT_SHAPE_MOTIFF_CATEGORY'] = product_shape_motiff_category_val

    except:
        pass
    

    
    
    index_count+=1





star
star
bear
3
5
6
heart
star
bear
heart
bear
bear
bear
2
star
2
3
star
aquarius
star
aries
heart
7
letter
letter
letter
letter
letter
letter
letter
letter
6
star
virgo
letter
letter
letter
letter
letter
letter
letter
letter
6
letter
letter
letter
letter
letter
letter
letter
letter
6
fishing
3
4
insect
animal
7
plant
3
5
star
taurus
letter
letter
letter
letter
letter
letter
letter
letter
6
heart
letter
letter
letter
letter
letter
letter
letter
letter
6
3
5
fishing
letter
letter
letter
letter
letter
letter
letter
letter
6
letter
letter
letter
letter
letter
letter
letter
letter
6
bear
letter
letter
letter
letter
letter
letter
letter
letter
6
bear
5
9
star
bear
3
5
6
bear
heart
6
bear
bear
6
flower
bear
bear
bear
bear
heart
flower
star
bear
bear
bear
bear
bear
bear
bear
bear
bear
fishing
bear
7
letter
letter
letter
letter
letter
letter
letter
letter
6
star
capricorn
star
leo
star
gemini
letter
letter
letter
letter
letter
letter
letter
letter
6
insect
animal
8
plant
letter
letter
letter


In [153]:
resp

[{'description': 'looking fun trendy look go iconic jewelry splash color lilac-colored elastic bracelet silver star part sweet doll collection versatile piece jewelry go anywhere you.tous sweet doll lilac-colored elastic bracelet silver vermeil star charm charm size 14 x 15 mm bracelet length 17 cm.lilac-colored elastic sweet doll bracelet silver vermeil star',
  'SKU': '212781640',
  'tags': {'CATEGORY_1': ['jewelry'],
   'CATEGORY_2': ['bracelet'],
   'CATEGORY_3': ['charm'],
   'RAW_METAL': ['silver'],
   'MAIN_MATERIAL': ['silver'],
   'SECONDARY_MATERIAL': ['silver'],
   'PRODUCT_SHAPE_MOTIFF_SUBCATEGORY': ['star'],
   'length': '17cm',
   'size': '14 x 15mm',
   'PRODUCT_SHAPE_MOTIFF_CATEGORY': 'celestial symbols'}},
 {'description': 'looking fun trendy look go iconic jewelry splash color blue elastic bracelet silver star part sweet doll collection versatile piece jewelry go anywhere you.tous sweet doll blue elastic bracelet sterling silver star charm charm size 14 x 15 mm bracel

In [10]:

GEMSTONE_NAME = ['diamond', 'fancy diamond', 'lab-created diamond', 'crystals', 'saltwater pearl', 'freshwater pearl', 'akoya-cultured pearl', 'biwa-cultured pearl', 'chinese-freshwater-cultured pearl', 'hybrid-cultured pearl', 'keshi-cultured pearl', 'mabe-cultured pearl', 'majorca-cultured pearl', 'natural-pearl', 'simulated-pearl', 'south-sea-baroque-cultured pearl', 'south-sea-cultured pearl', 'tahitian-cultured pearl', 'sapphire', 'ruby', 'emerald', 'sapphire', 'ruby', 'emerald', 'topaz', 'amethyst', 'aquamarine', 'citrine', 'garnet', 'onyx', 'opal', 'peridot', 'tanzanite', 'achroite', 'agate', 'akoya pearl', 'alexandrite', 'almandine\xa0garnet', 'amazonite', 'amber', 'amethyst', 'ametrine', 'ammolite', 'anatase', 'andalusite', 'andesine', 'andradite\xa0garnet', 'apatite', 'aquamarine', 'arkansas stone', 'axinite', 'azurite', 'barite', 'benitoite', 'beryl', 'bi-color\xa0tourmaline', 'bixbite', 'bloodstone', 'bowenite', 'calcite', 'californite', 'carnelian', 'chalcedony', 'chalybite', 'charoite', 'chrome diopside', 'chrome sphene', 'chrome\xa0tourmaline', 'chrysoberyl', 'chrysocolla', 'chrysoprase', 'citrine', 'clinohumite', 'conch pearl', 'copal', 'coral', 'cordierite', 'corundum', 'cyprine', 'danburite', 'datolite', 'demantoid\xa0garnet', 'diaspore', 'diopside', 'dioptase', 'dravite\xa0tourmaline', 'ekanite', 'enstatite', 'feldspar', 'fluorite', 'freshwater pearl', 'garnet', 'goldstone', 'goshenite', 'green beryl', 'greenovite', 'grossular', 'hackmanite', 'hauyne', 'heliodor', 'hiddenite', 'hydrogrossular\xa0garnet', 'idocrase', 'inclusion quartz', 'indicolite\xa0tourmaline', 'iolite', 'jade', 'jadeite', 'jasper', 'kornerupine', 'kunzite', 'kyanite', 'labradorite', 'lapiz lazuli', 'larimar', 'liddicoatite tourmaline', 'malachite', 'maw-sit-sit', 'maxixe\xa0beryl', 'melo melo pearl', 'moonstone', 'morganite', 'nephrite', 'onyx', 'opal', 'paraiba type\xa0tourmaline', 'pearl', 'pectolite', 'peridot', 'phenakite', 'poudretteite', 'praseolite\xa0(cordierite)', 'prasiolite\xa0(green quartz)', 'prasolite\xa0(chlorine group)', 'pyrope\xa0garnet', 'quahog', 'quartz', 'reisling\xa0beryl', 'rhodochrosite', 'rock crystal', 'rose quartz', 'rubellite\xa0tourmaline', 'sard', 'sardonyx', 'scapolite', 'schorl\xa0tourmaline', 'sea of cortez pearl', 'siderite', 'smoky quartz', 'sodalite', 'south sea\xa0pearl', 'spectrolite', 'spessartite\xa0garnet', 'sphalerite', 'sphene', 'spinel', 'spodumene', 'sunstone', 'tanzanite', 'tashmarine\xa0diopside', 'thulite', 'tigereye', 'titanite', 'topaz', 'tourmaline', 'tri-color\xa0tourmaline', 'true blue\xa0beryl', 'tsavorite\xa0garnet', 'turquoise', 'uvarovite', 'vanadium beryl', 'verdite\xa0tourmaline', 'vesuvianite', 'watermelon\xa0tourmaline', 'willemite', 'zincite', 'zircon', 'zoisite', 'zultanite', 'multi-color', 'topaz (lab-created)', 'amethyst (lab-created)', 'aquamarine (lab-created)', 'citrine (lab-created)', 'garnet (lab-created)', 'onyx (lab-created)', 'opal (lab-created)', 'peridot (lab-created)', 'tanzanite (lab-created)', 'achroite (lab-created)', 'agate (lab-created)', 'akoya pearl (lab-created)', 'alexandrite (lab-created)', 'almandine\xa0garnet (lab-created)', 'amazonite (lab-created)', 'amber (lab-created)', 'amethyst (lab-created)', 'ametrine (lab-created)', 'ammolite (lab-created)', 'anatase (lab-created)', 'andalusite (lab-created)', 'andesine (lab-created)', 'andradite\xa0garnet (lab-created)', 'apatite (lab-created)', 'aquamarine (lab-created)', 'arkansas stone (lab-created)', 'axinite (lab-created)', 'azurite (lab-created)', 'barite (lab-created)', 'benitoite (lab-created)', 'beryl (lab-created)', 'bi-color\xa0tourmaline (lab-created)', 'bixbite (lab-created)', 'bloodstone (lab-created)', 'bowenite (lab-created)', 'calcite (lab-created)', 'californite (lab-created)', 'carnelian (lab-created)', 'chalcedony (lab-created)', 'chalybite (lab-created)', 'charoite (lab-created)', 'chrome diopside (lab-created)', 'chrome sphene (lab-created)', 'chrome\xa0tourmaline (lab-created)', 'chrysoberyl (lab-created)', 'chrysocolla (lab-created)', 'chrysoprase (lab-created)', 'citrine (lab-created)', 'clinohumite (lab-created)', 'conch pearl (lab-created)', 'copal (lab-created)', 'coral (lab-created)', 'cordierite (lab-created)', 'corundum (lab-created)', 'cyprine (lab-created)', 'danburite (lab-created)', 'datolite (lab-created)', 'demantoid\xa0garnet (lab-created)', 'diaspore (lab-created)', 'diopside (lab-created)', 'dioptase (lab-created)', 'dravite\xa0tourmaline (lab-created)', 'ekanite (lab-created)', 'enstatite (lab-created)', 'feldspar (lab-created)', 'fluorite (lab-created)', 'freshwater pearl (lab-created)', 'garnet (lab-created)', 'goldstone (lab-created)', 'goshenite (lab-created)', 'green beryl (lab-created)', 'greenovite (lab-created)', 'grossular (lab-created)', 'hackmanite (lab-created)', 'hauyne (lab-created)', 'heliodor (lab-created)', 'hiddenite (lab-created)', 'hydrogrossular\xa0garnet (lab-created)', 'idocrase (lab-created)', 'inclusion quartz (lab-created)', 'indicolite\xa0tourmaline (lab-created)', 'iolite (lab-created)', 'jade (lab-created)']
GEMSTONE_CATEGORY = ['diamonds', 'diamonds', 'lab-created diamonds', 'non-precious', 'pearls', 'pearls', 'pearls', 'pearls', 'pearls', 'pearls', 'pearls', 'pearls', 'pearls', 'pearls', 'pearls', 'pearls', 'pearls', 'pearls', 'precious', 'precious', 'precious', 'precious (lab-created)', 'precious (lab-created)', 'precious (lab-created)', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)', 'semi-precious (lab-created)']

count = 'Diamond,Fancy Diamond,Lab-Created Diamond,Crystals,Saltwater Pearl,Freshwater Pearl,akoya-cultured Pearl,biwa-cultured Pearl,chinese-freshwater-cultured Pearl,hybrid-cultured Pearl,keshi-cultured Pearl,mabe-cultured Pearl,majorca-cultured Pearl,natural-pearl,simulated-pearl,south-sea-baroque-cultured Pearl,south-sea-cultured Pearl,tahitian-cultured Pearl,Sapphire,Ruby,Emerald,Topaz,Amethyst,Aquamarine,Citrine,Garnet,Onyx,Opal,Peridot,Tanzanite,Achroite,Agate,Akoya Pearl,Alexandrite,Almandine Garnet,Amazonite,Amber,Ametrine,Ammolite,Anatase,Andalusite,Andesine,Andradite Garnet,Apatite,Arkansas stone,Axinite,Azurite,Barite,Benitoite,Beryl,Bi-color tourmaline,Bixbite,Bloodstone,Bowenite,Calcite,Californite,Carnelian,Chalcedony,Chalybite,Charoite,Chrome Diopside,Chrome sphene,Chrome Tourmaline,Chrysoberyl,Chrysocolla,Chrysoprase,Clinohumite,Conch pearl,Copal,Coral,Cordierite,Corundum,Cyprine,Danburite,Datolite,Demantoid Garnet,Diaspore,Diopside,Dioptase,Dravite Tourmaline,Ekanite,Enstatite,Feldspar,Fluorite,Freshwater pearl,Goldstone,Goshenite,Green Beryl,Greenovite,Grossular,Hackmanite,Hauyne,Heliodor,Hiddenite,Hydrogrossular Garnet,Idocrase,Inclusion quartz,Indicolite Tourmaline,Iolite,Jade,Jadeite,Jasper,Kornerupine,Kunzite,Kyanite,Labradorite,Lapiz Lazuli,Larimar,Liddicoatite Tourmaline,Malachite,Maw-sit-sit,Maxixe Beryl,Melo melo pearl,Moonstone,Morganite,Nephrite,Paraiba type Tourmaline,Pearl,Pectolite,Phenakite,Poudretteite,Praseolite (cordierite),Prasiolite (green quartz),Prasolite (chlorine group),Pyrope Garnet,Quahog,Quartz,Reisling Beryl,Rhodochrosite,Rock Crystal,Rose Quartz,Rubellite Tourmaline,Sard,Sardonyx,Scapolite,Schorl Tourmaline,Sea of Cortez pearl,Siderite,Smoky Quartz,Sodalite,South Sea Pearl,Spectrolite,Spessartite Garnet,Sphalerite,Sphene,Spinel,Spodumene,Sunstone,Tashmarine Diopside,Thulite,Tigereye,Titanite,Tourmaline,Tri-color Tourmaline,True Blue Beryl,Tsavorite Garnet,Turquoise,Uvarovite,Vanadium Beryl,Verdite Tourmaline,Vesuvianite,Watermelon Tourmaline,Willemite,Zincite,Zircon,Zoisite,Zultanite,Multi-Color,Topaz (Lab-Created),Amethyst (Lab-Created),Aquamarine (Lab-Created),Citrine (Lab-Created),Garnet (Lab-Created),Onyx (Lab-Created),Opal (Lab-Created),Peridot (Lab-Created),Tanzanite (Lab-Created),Achroite (Lab-Created),Agate (Lab-Created),Akoya Pearl (Lab-Created),Alexandrite (Lab-Created),Almandine Garnet (Lab-Created),Amazonite (Lab-Created),Amber (Lab-Created),Ametrine (Lab-Created),Ammolite (Lab-Created),Anatase (Lab-Created),Andalusite (Lab-Created),Andesine (Lab-Created),Andradite Garnet (Lab-Created),Apatite (Lab-Created),Arkansas stone (Lab-Created),Axinite (Lab-Created),Azurite (Lab-Created),Barite (Lab-Created),Benitoite (Lab-Created),Beryl (Lab-Created),Bi-color tourmaline (Lab-Created),Bixbite (Lab-Created),Bloodstone (Lab-Created),Bowenite (Lab-Created),Calcite (Lab-Created),Californite (Lab-Created),Carnelian (Lab-Created),Chalcedony (Lab-Created),Chalybite (Lab-Created),Charoite (Lab-Created),Chrome Diopside (Lab-Created),Chrome sphene (Lab-Created),Chrome Tourmaline (Lab-Created),Chrysoberyl (Lab-Created),Chrysocolla (Lab-Created),Chrysoprase (Lab-Created),Clinohumite (Lab-Created),Conch pearl (Lab-Created),Copal (Lab-Created),Coral (Lab-Created),Cordierite (Lab-Created),Corundum (Lab-Created),Cyprine (Lab-Created),Danburite (Lab-Created),Datolite (Lab-Created),Demantoid Garnet (Lab-Created),Diaspore (Lab-Created),Diopside (Lab-Created),Dioptase (Lab-Created),Dravite Tourmaline (Lab-Created),Ekanite (Lab-Created),Enstatite (Lab-Created),Feldspar (Lab-Created),Fluorite (Lab-Created),Freshwater pearl (Lab-Created),Goldstone (Lab-Created),Goshenite (Lab-Created),Green Beryl (Lab-Created),Greenovite (Lab-Created),Grossular (Lab-Created),Hackmanite (Lab-Created),Hauyne (Lab-Created),Heliodor (Lab-Created),Hiddenite (Lab-Created),Hydrogrossular Garnet (Lab-Created),Idocrase (Lab-Created),Inclusion quartz (Lab-Created),Indicolite Tourmaline (Lab-Created),Iolite (Lab-Created),Jade (Lab-Created),Jadeite (Lab-Created),Jasper (Lab-Created),Kornerupine (Lab-Created),Kunzite (Lab-Created),Kyanite (Lab-Created),Labradorite (Lab-Created),Lapiz Lazuli (Lab-Created),Larimar (Lab-Created),Liddicoatite Tourmaline (Lab-Created),Malachite (Lab-Created),Maw-sit-sit (Lab-Created),Maxixe Beryl (Lab-Created),Melo melo pearl (Lab-Created),Moonstone (Lab-Created),Morganite (Lab-Created),Nephrite (Lab-Created),Paraiba type Tourmaline (Lab-Created),Pearl (Lab-Created),Pectolite (Lab-Created),Phenakite (Lab-Created),Poudretteite (Lab-Created),Praseolite (cordierite) (Lab-Created),Prasiolite (green quartz) (Lab-Created),Prasolite (chlorine group) (Lab-Created),Pyrope Garnet (Lab-Created),Quahog (Lab-Created),Quartz (Lab-Created),Reisling Beryl (Lab-Created),Rhodochrosite (Lab-Created),Rock Crystal (Lab-Created),Rose Quartz (Lab-Created),Rubellite Tourmaline (Lab-Created),Sard (Lab-Created),Sardonyx (Lab-Created),Scapolite (Lab-Created),Schorl Tourmaline (Lab-Created),Sea of Cortez pearl (Lab-Created),Siderite (Lab-Created),Smoky Quartz (Lab-Created),Sodalite (Lab-Created),South Sea Pearl (Lab-Created),Spectrolite (Lab-Created),Spessartite Garnet (Lab-Created),Sphalerite (Lab-Created),Sphene (Lab-Created),Spinel (Lab-Created),Spodumene (Lab-Created),Sunstone (Lab-Created),Tashmarine Diopside (Lab-Created),Thulite (Lab-Created),Tigereye (Lab-Created),Titanite (Lab-Created),Tourmaline (Lab-Created),Tri-color Tourmaline (Lab-Created),True Blue Beryl (Lab-Created),Tsavorite Garnet (Lab-Created),Turquoise (Lab-Created),Uvarovite (Lab-Created),Vanadium Beryl (Lab-Created),Verdite Tourmaline (Lab-Created),Vesuvianite (Lab-Created),Watermelon Tourmaline (Lab-Created),Willemite (Lab-Created),Zincite (Lab-Created),Zircon (Lab-Created),Zoisite (Lab-Created),Zultanite (Lab-Created),CZ Cubic zirconia (synthetic),Swarovski Crystals,Glass'


count= count.split()
print(len(count))
print(len(GEMSTONE_NAME))
print(len(GEMSTONE_CATEGORY))

GEMSTONE_NAME_STR = ''
for word in GEMSTONE_NAME:
    GEMSTONE_NAME_STR+=f"{word},"
    
print(GEMSTONE_NAME_STR)




263
263
263
diamond,fancy diamond,lab-created diamond,crystals,saltwater pearl,freshwater pearl,akoya-cultured pearl,biwa-cultured pearl,chinese-freshwater-cultured pearl,hybrid-cultured pearl,keshi-cultured pearl,mabe-cultured pearl,majorca-cultured pearl,natural-pearl,simulated-pearl,south-sea-baroque-cultured pearl,south-sea-cultured pearl,tahitian-cultured pearl,sapphire,ruby,emerald,sapphire,ruby,emerald,topaz,amethyst,aquamarine,citrine,garnet,onyx,opal,peridot,tanzanite,achroite,agate,akoya pearl,alexandrite,almandine garnet,amazonite,amber,amethyst,ametrine,ammolite,anatase,andalusite,andesine,andradite garnet,apatite,aquamarine,arkansas stone,axinite,azurite,barite,benitoite,beryl,bi-color tourmaline,bixbite,bloodstone,bowenite,calcite,californite,carnelian,chalcedony,chalybite,charoite,chrome diopside,chrome sphene,chrome tourmaline,chrysoberyl,chrysocolla,chrysoprase,citrine,clinohumite,conch pearl,copal,coral,cordierite,corundum,cyprine,danburite,datolite,demantoid garnet,d

In [49]:
''' Step 7 '''
import pandas as pd
pd.read_json(json.dumps(resp)).to_csv('jun 9 Results.csv')

In [97]:
with open("result.json", "w") as outfile:
    json.dump(resp, outfile)

In [76]:
resp

[{'description': 'looking fun trendy look go iconic jewelry splash color lilac-colored elastic bracelet silver star part sweet doll collection versatile piece jewelry go anywhere you.tous sweet doll lilac-colored elastic bracelet silver vermeil star charm charm size 14 x 15 mm bracelet length 17 cm.lilac-colored elastic sweet doll bracelet silver vermeil star',
  'tags': {'CATEGORY_1': ['jewelry'],
   'CATEGORY_2': ['bracelet'],
   'MAIN_MATERIAL': ['silver'],
   'SECONDARY_MATERIAL': ['silver'],
   'SIZE_SYSTEM': ['mm'],
   'RAW_METAL': ['silver'],
   'PLATING_METAL': ['silver'],
   'length': '17cm',
   'size': '14 x 15mm'}},
 {'description': 'looking fun trendy look go iconic jewelry splash color blue elastic bracelet silver star part sweet doll collection versatile piece jewelry go anywhere you.tous sweet doll blue elastic bracelet sterling silver star charm charm size 14 x 15 mm bracelet length 17 cm.blue elastic sweet doll bracelet silver star',
  'tags': {'CATEGORY_1': ['jewelry'

In [83]:
cheeck =['gold', 'silverd']
if 'silver' in cheeck:
    print("yess")

In [77]:
 import csv

 with open('client.csv', 'w', newline='') as file:
     writer = csv.writer(file)
     writer.writerows(resp)

In [7]:
resp

[{'description': 'looking fun trendy look go iconic jewelry splash color lilac-colored elastic bracelet silver star part sweet doll collection versatile piece jewelry go anywhere you.tous sweet doll lilac-colored elastic bracelet silver vermeil star charm charm size 14 x 15 mm bracelet length 17 cm.lilac-colored elastic sweet doll bracelet silver vermeil star',
  'SKU': '212781640',
  'tags': {'CATEGORY_1': ['jewelry'],
   'CATEGORY_2': ['bracelet'],
   'CATEGORY_3': ['charm'],
   'RAW_METAL': ['silver'],
   'MAIN_MATERIAL': ['silver'],
   'SECONDARY_MATERIAL': ['silver'],
   'PRODUCT_SHAPE_MOTIFF_SUBCATEGORY': ['star'],
   'length': '17cm',
   'size': '14 x 15mm',
   'PRODUCT_SHAPE_MOTIFF_CATEGORY': 'celestial symbols'}},
 {'description': 'looking fun trendy look go iconic jewelry splash color blue elastic bracelet silver star part sweet doll collection versatile piece jewelry go anywhere you.tous sweet doll blue elastic bracelet sterling silver star charm charm size 14 x 15 mm bracel

In [293]:
import pandas as pd
pd.read_json(json.dumps(resp)).to_csv('19_april_results.csv')

In [148]:
key='GEMSTONE_1_CUT'
key=key.split(',')
print(key)
per=fuzz.ratio('GEMSTONE_', key)
if per==67:
    key=key.split('_')
    

['GEMSTONE_1_CUT']
64


In [271]:

datavvv='wear favorite initial silver silver vermeil letter c medallion pendant tous alphabet collection carry personal piece jewelry always.tous alphabet medallion pendant sterling silver letter c silver vermeil medallion diameter 15 mm letter size 6 mm item include chain.silver silver vermeil'


words = list(datavvv.split(" "))

for word in words:
    if word == 'diameter':
        print(words.index(word))
        print(words[words.index(word):(words.index(word)+3)])
        li=words[words.index(word):(words.index(word)+3)]
        
        if 'mm' in li:
            print(li[1]+" "+li[2])














28
['diameter', '15', 'mm']
15 mm


In [22]:
import pandas as pd

# Read Excel file into a pandas dataframe
df = pd.read_excel('input_file.xlsx')

# Write dataframe to a CSV file
df.to_csv('output_file.csv', index=False)


1
2
3
4
5


In [11]:
words = ['star', 'lilac-colored', 'iconic', 'size', 'looking', 'x', 'trendy', 'bracelet', '17', 'splash', 'vermeil', 'you.tous', '14', 'look', 'piece', 'jewelry', 'versatile', 'anywhere', 'sweet', 'collection', 'go', 'part', 'length', 'charm', 'color', 'fun', 'doll', 'elastic', 'cm.lilac-colored', 'silver', 'mm', '15']

for word in words:
    print(word)








star
lilac-colored
iconic
size
looking
x
trendy
bracelet
17
splash
vermeil
you.tous
14
look
piece
jewelry
versatile
anywhere
sweet
collection
go
part
length
charm
color
fun
doll
elastic
cm.lilac-colored
silver
mm
15


In [12]:

final_json = dict()
description = "le minimalist jewelry full style hoop earring gold\nblue sapphire diamond basic collection symbol\ntimelessness good taste combine earring create\nsuper-trendy look tous basic hoop earring 18kt\nyellow gold blue sapphire 0.03 ct h/si brilliant\ncut diamond thickness 2 mm \u00f88 mm hoop clasp\nproduction technique castinggold tous basic hoop\nearring blue sapphire diamond"
final_json["description"] = description
final_json["SKU"] = chunk['SKU']
#         values  ['Jewelry']
#         category  CATEGORY_1
#         words  ['timelessness', 'symbol', '18kt', 'ø8', 'full', 'basic', 'production', 'cut', 'taste', 'castinggold', '0.03', 'sapphire', 'gold', '2', 'brilliant', 'earring', 'minimalist', 'tous', 'collection', 'technique', 'diamond', 'super-trendy', 'look', 'ct', 'h/si', 'yellow', 'le', 'hoop', 'create', 'blue', 'clasp', 'style', 'jewelry', 'thickness', 'good', 'combine', 'mm']
#         item  ('jewelry',)
#         word  timelessness
for category in category_dict:
    values = category_dict[category]
    words = list(set(description.split(" ")))
    words = [word.lower() for word in words]
    for item in category_dict[category]:
        item = str(item)
        item = preprocess(item)

        for finder in words:
            match_ratio = fuzz.ratio(item, finder)
            if ( match_ratio > 90):
                GEMSTONE_Token = True
                if category in final_output:
                    per=fuzz.ratio('GEMSTONE_', category)
                    if per >= 50:
                        category_splited=category.split('_')
#                                 /* [0]GEMSTONE [1]4 [2]NAME */
#                                 /* GEMSTONE_1_NAME */
                        key=f"GEMSTONE_{category_splited[1]}_{category_splited[2]}"
#                                 /*GEMSTONE_4_Name*/
                        GEMSTONE_Token = True
                        if int(category_splited[1]) > 1:
                            if final_output is not None:
                                for gem_num in range(1,int(category_splited[1])):
                                    if final_output.get(f"GEMSTONE_{gem_num}_{category_splited[2]}") is not None:
                                        if item in final_output.get(f"GEMSTONE_{gem_num}_{category_splited[2]}"):
                                            GEMSTONE_Token = False
                                            continue
                                        else:
                                            GEMSTONE_Token = False
                                            continue
                                    else:
                                        GEMSTONE_Token = False
                                        continue
                    if GEMSTONE_Token:                                        
                        final_output[category].append(item)
                else:
                    GEMSTONE_Token = True
                    per=fuzz.ratio('GEMSTONE_', category)

                    if per >= 50:
#                                 /* [0]GEMSTONE [1]4 [2]NAME */
                        category_splited=category.split('_')
#                                 /*GEMSTONE_4_Name*/
                        key=f"GEMSTONE_{category_splited[1]}_{category_splited[2]}"
                        try:
                            if int(category_splited[1]) > 1:
                                if final_output is not None:
                                    for gem_num in range(1,int(category_splited[1])):
                                        if final_output.get(f"GEMSTONE_{gem_num}_{category_splited[2]}") is not None:
                                            if item in final_output.get(f"GEMSTONE_{gem_num}_{category_splited[2]}"):
                                                GEMSTONE_Token = False
                                                continue
                                            else:
                                                GEMSTONE_Token = False
                                                continue
                                        else:
                                            GEMSTONE_Token = False
                                            continue
                        except:
                            pass
                    if GEMSTONE_Token:
                        final_output[category] = [item]

            if category in final_output and (finder == 'vermeil' or finder == 'Vermeil'):
                final_output['Main_Material'] = "Silver"
                final_output['Secondary_Material']= "Gold" 
                final_output['Raw_Metal'] = "Silver"  
                final_output['Plating_Metal'] = "Gold"
            elif category in final_output and (finder == 'Silver Vermeil' or finder == 'silver vermeil'):
                final_output['Main_Material'] = "Silver"
                final_output['Secondary_Material']= "Gold" 
                final_output['Raw_Metal'] = "Silver"  
                final_output['Plating_Metal'] = "Gold"
            elif finder == 'vermeil' or finder == 'Vermeil' or finder == 'Silver Vermeil' or finder == 'silver vermeil':
                final_output['Main_Material'] = "Silver"
                final_output['Secondary_Material']= "Gold" 
                final_output['Raw_Metal'] = "Silver"  
                final_output['Plating_Metal'] = "Gold"
            if finder == 'diameter':
                try:
                    diameter_li=words[words.index(finder):(words.index(finder)+3)]
                    if 'mm' in diameter_li:
                        final_output['PRODUCT_OUTTER_DIAMETER'] = diameter_li[1]+" "+diameter_li[2]
                except:
                    pass
            if finder == 'Gold Plated' or finder == 'Gold Filled':
                if ('Silver' in description or 'silver' in description) and  category in final_output: 
                    final_output['Main_Material'] = "Silver"
                    final_output['Secondary_Material'] = "Gold" 
                    final_output['Raw_Metal'] = "Silver"
                    final_output['Plating_Metal'] = "Gold"
                elif ('Silver' in description or 'silver' in description): 
                    final_output['Main_Material'] = "Silver"
                    final_output['Secondary_Material'] = "Gold" 
                    final_output['Raw_Metal'] = "Silver"
                    final_output['Plating_Metal'] = "Gold"
                elif ('Brass' in description or 'brass' in description) and  category in final_output: 
                    final_output['Main_Material'] = "Brass"
                    final_output['Secondary_Material'] = "Gold"
                    final_output['Raw_Metal'] = "Brass"
                    final_output['Plating_Metal'] = "Gold"
                elif ('Brass' in description or 'brass' in description): 
                    final_output['Main_Material'] = "Brass"
                    final_output['Secondary_Material'] = "Gold" 
                    final_output['Raw_Metal'] = "Brass"
                    final_output['Plating_Metal'] = "Gold"
                elif ('Steel' in description or 'steel' in description) and  category in final_output: 
                    final_output['Main_Material'] = "Steel" 
                    final_output['Secondary_Material'] = "Gold" 
                    final_output['Raw_Metal'] = "Steel" 
                    final_output['Plating_Metal'] = "Gold"
                elif ('Steel' in description or 'steel' in description): 
                    final_output['Main_Material'] = "Steel"
                    final_output['Secondary_Material'] = "Gold" 
                    final_output['Raw_Metal'] = "Steel"
                    final_output['Plating_Metal'] = "Gold"
            if category in final_output and (finder == 'Sterling Silver' or finder == 'sterling silver'):
                final_output['Main_Material'] = "Silver"
                final_output['Raw_Material_Purity'] = "925"
            elif finder == 'Sterling Silver' or finder == 'sterling silver':
                final_output['Main_Material'] = "Silver"  
                final_output['Raw_Material_Purity'] = "925"
            if final_output.get('Secondary_Material') and  final_output['Secondary_Material'] == final_output['Main_Material']:
                final_output['Raw_Metal'] = final_output['Main_Material']


NameError: name 'chunk' is not defined